In [1]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import sys, time
import numpy as np
sys.path.insert(0, '../')
import tensorflow as tf
from library.datasets import cifar10
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from library.utils import file_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

None


In [2]:
total_time = 0

In [3]:
cifar10_dataset = cifar10.CIFAR10(one_hot_encode=True, num_images=1.0, image_mode='rgb_float',
                                  train_validate_split=None, endian='little')
cifar10_dataset.load_data(train=True, test=True, data_directory='./datasets/cifar10/')

Loading CIFAR 10 Dataset
MD5sum of the file: ./datasets/cifar10/cifar-10.tar.gz is verified
Loading 50000 train images
Loading CIFAR 10 Training Dataset
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_1
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_2
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_3
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_4
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_5
Loading 10000 test images
Loading CIFAR 10 Test Dataset
Unpickling test file: ./datasets/cifar10/cifar-10-batches/test_batch
Reading unpicked test file: ./datasets/cifar10/cifar-10-batches/test_batch
Loaded CIFAR 10 Dataset in 9.2391 seconds


True

In [4]:
X = cifar10_dataset.train.images
Y = cifar10_dataset.train.one_hot_labels
X_test = cifar10_dataset.test.images
Y_test = cifar10_dataset.test.one_hot_labels

X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size=0.2, random_state=1)


In [5]:
exp_no = 1
file_no = 20
dataset = 'cifar10'


In [6]:
model_dir='./logs/model1'

In [7]:
log_dir = '../logs/' + dataset + '/' + str(file_no).zfill(2) + '_tf_cnn/exp_no_' + str(exp_no) + '/'
#ck_path = '../ckpts/' + 'ckpt_tf_cnn/exp_no_' + str(exp_no) + '/'+'model.tfl.ckpt'
#cptfile = model.tfl.ckpt
ck_path = log_dir + 'model.tfl.ckpt'
log_file = log_dir + 'tflearncnn_classifier.ckpt'
print('Writing tensorboard logs to %s' %log_file)
print('view logs by running tensorboard: ', end='')
print('\"tensorboard --logdir=\'./10_tf_cnn/\' --port 61111\"')

Writing tensorboard logs to ../logs/cifar10/20_tf_cnn/exp_no_1/tflearncnn_classifier.ckpt
view logs by running tensorboard: "tensorboard --logdir='./10_tf_cnn/' --port 61111"


In [8]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [9]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [10]:
network = input_data(shape=[None, 32, 32, 3],data_preprocessing=img_prep,data_augmentation=img_aug)
#network = input_data(shape=[None, 32, 32, 3])

In [11]:
# 0.80589
# network = conv_2d(network, 32, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 128, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = conv_2d(network, 512, 3, activation='relu')
# network = max_pool_2d(network, 2)
# network = fully_connected(network, 1024, activation='relu')
# network = dropout(network, 0.5)
# network = fully_connected(network, 10, activation='softmax')
# network = regression(network, optimizer='adam',
#                      loss='categorical_crossentropy',
#                      learning_rate=0.001)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 512, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 2048, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [12]:
start = time.time()
model = tflearn.DNN(network, tensorboard_verbose=1,tensorboard_dir=log_file,checkpoint_path=ck_path)
#model.load(ck_path)

model.fit(X_train, Y_train, n_epoch=5, shuffle=True,snapshot_epoch=True, validation_set=(X_validate, Y_validate),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')
end = time.time()
print(end-start)

Training Step: 2084  | total loss: 0.85927 | time: 127.289s
| Adam | epoch: 005 | loss: 0.85927 - acc: 0.6946 -- iter: 39936/40000
Training Step: 2085  | total loss: 0.86514 | time: 134.203s
| Adam | epoch: 005 | loss: 0.86514 - acc: 0.6949 | val_loss: 0.79440 - val_acc: 0.7192 -- iter: 40000/40000
--
INFO:tensorflow:/home/ml/Documents/MLp/zip/logs/cifar10/20_tf_cnn/exp_no_1/model.tfl.ckpt-2085 is not in all_model_checkpoint_paths. Manually adding it.
686.200014591217


In [13]:
# labels = model.predict_label(X_test)

In [14]:
# labels1 = numpy.floor(numpy.divide(labels,numpy.amax(labels,axis=0)))
# labels2 = numpy.argmax(labels1, axis=1, out=None)
# print(labels2)
# print(cifar10_dataset.test.class_labels)

In [15]:
# result = accuracy_score(cifar10_dataset.test.class_labels, labels2, normalize=True)
# print("accuracy",result)

In [16]:
model.evaluate(cifar10_dataset.test.images, cifar10_dataset.test.one_hot_labels)

[0.7248]